In [1]:
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors

In [2]:
wv_skipgram_w5 = KeyedVectors.load_word2vec_format("results/models/m_skipgram_w5.vec")
wv_skipgram_w10 = KeyedVectors.load_word2vec_format("results/models/m_skipgram_w10.vec")
wv_cbow_w5 = KeyedVectors.load_word2vec_format("results/models/m_cbow_w5.vec")

# https://fasttext.cc/docs/en/crawl-vectors.html#models
wv_pretrained = KeyedVectors.load_word2vec_format("results/models/cc.be.300.vec")

In [3]:
def apply_func(func, index = None):
    map_func = lambda x: ": ".join(map(str,x)) if type(x) == tuple else x
    
    table = {
        "skipgram w5": map(map_func, func(wv_skipgram_w5, "skipgram w5")),
        "skipgram w10": map(map_func, func(wv_skipgram_w10, "skipgram w10")),
        "cbow w5": map(map_func, func(wv_cbow_w5, "cbow w5")),
        "pretrained": map(map_func, func(wv_pretrained, "pretrained"))
    }
    
    return pd.DataFrame(table, index)

In [4]:
apply_func(
    lambda x, _: [len(x.vectors)],
    index=["Number of vectors"]
)

,skipgram w5,skipgram w10,cbow w5,pretrained
Number of vectors,114491,114491,114491,1714011


# Outliers detection

In [5]:
outliers_stats = {
    "skipgram w5": 0,
    "skipgram w10": 0,
    "cbow w5": 0,
    "pretrained": 0
}


def detect_outliers(wv: KeyedVectors, model_name: str):
    column = []
    
    for outlier in cluster_outliers:
        predicted_outlier = wv.doesnt_match(
            cluster_elements + [outlier]
        )
        
        if predicted_outlier == outlier:
            column.append("+")
        else:
            column.append("- " + predicted_outlier)
            
    column.append(f"{column.count('+')}/8")
    outliers_stats[model_name] += column.count('+')
    
    return column

In [6]:
cluster_elements = ["чырвоны", "белы", "сіні", "блакітны", "зялёны", "чорны", "жоўты", "карычневы"]
cluster_outliers = ["мастак", "вясёлка", "гарбата", "аловак", "неба", "плот", "эмоцыя", "старажытнасць"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
мастак,+,+,+,+
вясёлка,- белы,- чырвоны,+,+
гарбата,+,- чырвоны,+,+
аловак,+,+,+,+
неба,+,+,+,+
плот,+,+,+,+
эмоцыя,+,+,+,+
старажытнасць,+,+,+,+
Correct predictions,7/8,6/8,8/8,8/8


In [7]:
cluster_elements = ["сабака", "кот", "леў", "мыш", "ліс", "воўк", "мядзведзь", "заяц"]
cluster_outliers = ["лес", "камень", "пыса", "хвост", "заапарк", "зграя", "тэхніка", "мастацтва"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
лес,+,+,+,- мыш
камень,+,+,+,+
пыса,- сабака,- сабака,- сабака,- мыш
хвост,+,+,- сабака,- мыш
заапарк,- сабака,+,- сабака,- мыш
зграя,- сабака,+,- сабака,- мыш
тэхніка,+,+,+,+
мастацтва,+,+,+,+
Correct predictions,5/8,7/8,4/8,3/8


In [8]:
cluster_elements = ["студзень", "сакавік", "май", "ліпень", "жнівень", "верасень", "лістапад", "снежань"]
cluster_outliers = ["серада", "зіма", "дата", "год", "астралогія", "нараджэнне", "мяч", "папера"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
серада,+,+,+,+
зіма,+,+,+,+
дата,+,+,+,+
год,+,+,+,+
астралогія,+,+,+,+
нараджэнне,+,+,+,+
мяч,+,+,+,+
папера,+,+,+,+
Correct predictions,8/8,8/8,8/8,8/8


In [9]:
cluster_elements = ["галава", "рука", "нага", "тулава", "шыя", "палец", "спіна", "плячо"]
cluster_outliers = ["медыцына", "мышца", "шапка", "адзенне", "футбол", "праца", "смоўж", "планета"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
медыцына,+,+,+,+
мышца,- галава,- галава,+,- палец
шапка,- галава,- галава,- галава,+
адзенне,- галава,+,+,+
футбол,+,+,+,+
праца,+,+,+,+
смоўж,- галава,+,+,+
планета,+,+,+,+
Correct predictions,4/8,6/8,7/8,7/8


In [10]:
cluster_elements = ["масленіца", "купалле", "вялікдзень", "спас", "сёмуха", "вадохрышча", "радаўніца", "дажынкі"]
cluster_outliers = ["падарунак", "святочнасць", "абрад", "прымха", "прысмак", "сузор'е", "графін", "воск"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
падарунак,+,+,- вадохрышча,+
святочнасць,+,+,- вадохрышча,- спас
абрад,+,+,- вадохрышча,- спас
прымха,- вадохрышча,- спас,- вадохрышча,+
прысмак,+,+,- вадохрышча,- спас
сузор'е,- вадохрышча,- спас,- вадохрышча,- спас
графін,- вадохрышча,- спас,- вадохрышча,+
воск,+,- спас,- вадохрышча,+
Correct predictions,5/8,4/8,0/8,4/8


In [11]:
cluster_elements = ["вуліца", "дом", "парк", "плошча", "праспект", "сквер", "завулак", "бульвар"]
cluster_outliers = ["вёска", "краіна", "мэр", "транспарт", "турыст", "мапа", "кава", "крыж"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
вёска,+,+,+,+
краіна,+,+,+,+
мэр,- дом,- дом,- дом,+
транспарт,+,+,- дом,+
турыст,- дом,- вуліца,- дом,+
мапа,+,+,- дом,+
кава,+,+,+,+
крыж,+,+,+,+
Correct predictions,6/8,6/8,4/8,8/8


In [12]:
cluster_elements = ["аўтобус", "тралейбус", "машына", "трамвай", "метро", "ровар", "самакат", "матацыкл"]
cluster_outliers = ["дарога", "кола", "горад", "квіток", "маршрут", "хуткасць", "груша", "месяц"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
дарога,+,+,- самакат,+
кола,+,+,- самакат,+
горад,+,+,- самакат,+
квіток,- машына,+,- самакат,- метро
маршрут,- машына,+,- самакат,- метро
хуткасць,+,+,- самакат,+
груша,+,+,- самакат,+
месяц,+,+,- самакат,+
Correct predictions,6/8,8/8,0/8,6/8


In [13]:
cluster_elements = ["рондаль", "патэльня", "чайнік", "духоўка", "пліта", "тарка", "халадзільнік", "скавародка"]
cluster_outliers = ["страва", "кухар", "пакой", "вада", "кватэра", "канапа", "марс", "трава"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
страва,+,+,- духоўка,- пліта
кухар,+,+,- духоўка,+
пакой,+,+,- духоўка,+
вада,+,+,- духоўка,+
кватэра,+,+,- духоўка,+
канапа,+,+,- духоўка,- пліта
марс,- пліта,- пліта,- духоўка,+
трава,+,+,- духоўка,+
Correct predictions,7/8,7/8,0/8,6/8


In [14]:
for col, val in outliers_stats.items():
    outliers_stats[col] = [f"{val}/64", val / 64 * 100]

pd.DataFrame(
    data=outliers_stats,
    index=["Number of correctly predicted", "% of correctly predicted"]
).T

,Number of correctly predicted,% of correctly predicted
skipgram w5,48/64,75.0
skipgram w10,52/64,81.25
cbow w5,31/64,48.4375
pretrained,50/64,78.125
